In [33]:
# Import the clr module, which provides the .NET Common Language Runtime (CLR) functionality for Python
import clr

# Import the System.IO module, which provides access to the input/output (I/O) operations on files and directories
import System.IO

# Import the System module, which provides access to common types and functionality in the .NET framework
import System

# Import the pythoncom module, which provides Python bindings for COM (Component Object Model) services
import pythoncom

# Import the Directory, Path, and File classes from the System.IO module, which provide methods for working with directories and files
from System.IO import Directory, Path, File

# Import the String and Environment classes from the System module, which provide methods for working with strings and environment variables
from System import String, Environment

# Initialize the COM components
pythoncom.CoInitialize()

# Set the path to the DWSIM installation folder
dwSimPath = r"C:\Users\viraj\AppData\Local\DWSIM\\"

# Add references to the DWSIM libraries using the clr module
# The AddReference method is used to add a reference to a .NET assembly
# The path to the assembly is constructed using the dwSimPath variable and the name of the assembly file
# Each assembly provides a specific set of functionality for DWSIM
clr.AddReference(dwSimPath + "CapeOpen.dll")
clr.AddReference(dwSimPath + "DWSIM.Automation.dll")
clr.AddReference(dwSimPath + "DWSIM.Interfaces.dll")
clr.AddReference(dwSimPath + "DWSIM.GlobalSettings.dll")
clr.AddReference(dwSimPath + "DWSIM.SharedClasses.dll")
clr.AddReference(dwSimPath + "DWSIM.Thermodynamics.dll")
clr.AddReference(dwSimPath + "DWSIM.UnitOperations.dll")
clr.AddReference(dwSimPath + "DWSIM.Inspector.dll")
clr.AddReference(dwSimPath + "System.Buffers.dll")

# Import specific classes from the DWSIM libraries using the "from...import" syntax
# The classes are used in the subsequent code to access specific functionality provided by DWSIM
from DWSIM.Interfaces.Enums.GraphicObjects import ObjectType
from DWSIM.Thermodynamics import Streams, PropertyPackages
from DWSIM.UnitOperations import UnitOperations
from DWSIM.Automation import Automation3
from DWSIM.GlobalSettings import Settings
from System import Array
# Set the current directory to the DWSIM installation folder using the Directory class from the System.IO module
# This is necessary so that the DWSIM libraries can be located and imported correctly
Directory.SetCurrentDirectory(dwSimPath)

In [34]:
# Create an instance of the Automation3 class from the DWSIM.Automation module
# This class provides methods for automating tasks in DWSIM, such as creating and manipulating flowsheets
interf = Automation3()

In [35]:
# Creates a flowsheet
sim = interf.CreateFlowsheet()

In [36]:
# Add Compounds
cnames = ["Water"]
sim.AddCompound("Water")

In [37]:
one = sim.AddFlowsheetObject('Material Stream','1')
two = sim.AddFlowsheetObject('Material Stream','2')
three = sim.AddFlowsheetObject('Material Stream','3')
SPL_1 = sim.AddFlowsheetObject('Stream Splitter','SPL-1')

In [38]:
one = one.GetAsObject()
two = two.GetAsObject()
three = three.GetAsObject()
SPL_1 = SPL_1.GetAsObject()

In [39]:
sim.ConnectObjects(one.GraphicObject , SPL_1.GraphicObject, -1, -1)
sim.ConnectObjects(SPL_1.GraphicObject , two.GraphicObject, -1, -1)
sim.ConnectObjects(SPL_1.GraphicObject , three.GraphicObject, -1, -1)

In [40]:
sim.AutoLayout()

In [41]:
one.SetOverallComposition(Array[float]([1]))
one.SetTemperature(350.0) # K
one.SetPressure(101325.0) # Pa
one.SetMassFlow(1.0) # kg/s

'1: mass flow set to 1 kg/s'

In [42]:
# property package
Steam_table = sim.CreateAndAddPropertyPackage("Steam Tables (IAPWS-IF97)")

In [43]:
# Calc Modes
SPL_1.OperationMode.SplitRatios
SPL_1.Ratios[0] = 0.5
SPL_1.Ratios[1] = 0.5

In [44]:
# request a calculation
errors = interf.CalculateFlowsheet4(sim)
list(errors)

[]

In [45]:
# save file

fileNameToSave = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), r"D:\08 Linked In\05 Github\DWSim-Automation-Repo\00 FlowSheet Automation\02 Splitter Automation\02 Splitter.dwxmz")

interf.SaveFlowsheet(sim, fileNameToSave, True)

In [46]:
# save the pfd to an image and display it

clr.AddReference(dwSimPath + "SkiaSharp.dll")
clr.AddReference("System.Drawing")

from SkiaSharp import SKBitmap, SKImage, SKCanvas, SKEncodedImageFormat
from System.IO import MemoryStream
from System.Drawing import Image
from System.Drawing.Imaging import ImageFormat

PFDSurface = sim.GetSurface()

imgwidth = 1024
imgheight = 768

bmp = SKBitmap(imgwidth, imgheight)
canvas = SKCanvas(bmp)
PFDSurface.Center(imgwidth, imgheight)
PFDSurface.ZoomAll(imgwidth, imgheight)
PFDSurface.UpdateCanvas(canvas)
d = SKImage.FromBitmap(bmp).Encode(SKEncodedImageFormat.Png, 100)
str = MemoryStream()
d.SaveTo(str)
image = Image.FromStream(str)
imgPath = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), r"D:\08 Linked In\05 Github\DWSim-Automation-Repo\00 FlowSheet Automation\02 Splitter Automation\02 Splitter.png")
image.Save(imgPath, ImageFormat.Png)
str.Dispose()
canvas.Dispose()
bmp.Dispose()

from PIL import Image

im = Image.open(imgPath)
im.show()